In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [2]:
df = pd.read_pickle('merged_df.pkl')
print(df.shape)
print()
df.head()

(625431, 8)



,asin,overall,reviewText,style,reviewerID,unixReviewTime,format,brand
0,4126895493,5.0,Great headphones. It's just the cord is too sh...,{'Color:': ' Blue W/Mic'},A38RQFVQ1AKJQQ,1424563200,None,HeadGear
1,4126895493,5.0,Really like these headphone. Wanted something...,{'Color:': ' Blue Zebra W/Mic'},A299MRB9O6GWDE,1494201600,None,HeadGear
2,4126895493,1.0,Wire to headphone broke off in less than a mon...,{'Color:': ' Blue W/Mic'},A3ACFC6DQQLIQT,1478304000,None,HeadGear
3,4126895493,3.0,Very good,{'Color:': ' Green'},A36BC0YFDBNB5X,1474675200,None,HeadGear
4,4126895493,1.0,Currently returning this product because the s...,{'Color:': ' Violet Purple'},A212PQ0HQPNNWM,1468713600,None,HeadGear


In [3]:
df.dtypes

asin               object
overall           float64
reviewText         object
style              object
reviewerID         object
unixReviewTime      int64
format             object
brand              object
dtype: object

In [4]:
df.isnull().sum()

asin                   0
overall                0
reviewText            96
style                  0
reviewerID             0
unixReviewTime         0
format            620677
brand                  0
dtype: int64

In [3]:
df = df[['asin', 'overall', 'reviewText', 'style', 'reviewerID', 'unixReviewTime', 'brand']]

In [6]:
type(df['style'][0])

str

## 4. Obtain the Descriptive Statistics of the product as : -

In [7]:
# a. Number of Reviews.
df['reviewText'].dropna().count()

625335

In [8]:
# b. Average Rating Score.
df['overall'].mean()

4.108317304386895

In [9]:
# c. Number of Unique Products.
unique_counts = {}
import json
for value in df['style'].dropna():
    str_value = json.dumps(value)  
    if str_value in unique_counts:
        unique_counts[str_value] += 1
    else:
        unique_counts[str_value] = 1

print("Number of unique products:", len(unique_counts))

Number of unique products: 5885


In [10]:
# d. Number of Good Ratings.
threshold = 3
good_ratings_count = df[df['overall'] >= threshold].shape[0]
print("Number of good Ratings:", good_ratings_count)

Number of good Ratings: 535699


In [11]:
# e. Number of Bad Ratings.
threshold = 3
bad_ratings_count = df[df['overall'] < threshold].shape[0]
print("Number of Bad Ratings:", bad_ratings_count)

Number of Bad Ratings: 89732


In [12]:
# f. Number of Reviews corresponding to each Rating.
df_rating_review = df[['overall', 'reviewText']]
print(df_rating_review.shape)
df_rating_review = df_rating_review.dropna().reset_index(drop= True)
print(df_rating_review.shape)
df_rating_review.groupby(['overall']).size().reset_index(name='count')

(625431, 2)
(625335, 2)


,overall,count
0,1.0,49074
1,2.0,40652
2,3.0,60760
3,4.0,117862
4,5.0,356987


## 5. Preprocess the Text

In [13]:
df_rating_review.isnull().sum()

overall       0
reviewText    0
dtype: int64

In [14]:
df_rating_review.head()

,overall,reviewText
0,5.0,Great headphones. It's just the cord is too sh...
1,5.0,Really like these headphone. Wanted something...
2,1.0,Wire to headphone broke off in less than a mon...
3,3.0,Very good
4,1.0,Currently returning this product because the s...


In [15]:
df_rating_review.dtypes

overall       float64
reviewText     object
dtype: object

In [17]:
#! pip install inflect

In [18]:
import re
import unicodedata
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import inflect

lemmatizer = WordNetLemmatizer()
p = inflect.engine()

# a. Removing the HTML Tags.
def remove_html_tags(text):
    clean_text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    return clean_text

# b. Removing accented characters.
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

# c. Expanding Acronyms 
acronyms = {
    "IIRC": "If I Recall Correctly",
    "IMO": "In My Opinion",
    "YMMV": "Your Mileage May Vary",
    "TL;DR": "Too Long; Didn't Read",
    "IMHO": "In My Humble Opinion",
    "FWIW": "For What It's Worth",
    "AFAIK": "As Far As I Know",
    "OP": "Original Poster",
    "ETA": "Estimated Time of Arrival",
    "OT": "Off-Topic",
    "NIB": "New In Box",
    "BNIB": "Brand New In Box",
    "BNWT": "Brand New With Tags",
    "BNIP": "Brand New In Packaging",
    "EUC": "Excellent Used Condition",
    "NWOT": "New Without Tags",
    "FS": "For Sale",
    "FT": "For Trade",
    "ISO": "In Search Of",
    "NWT": "New With Tags",
    "HTF": "Hard To Find",
    "BN": "Brand New",
    "VGC": "Very Good Condition",
    "VGUC": "Very Good Used Condition",
    "LN": "Like New",
    "NOS": "New Old Stock",
    "NIP": "New In Package",
    "OBO": "Or Best Offer",
    "PP": "PayPal",
    "PM": "Private Message",
    "NWT": "New With Tags",
    "NIB": "New In Box",
    "MIB": "Mint In Box",
    "MIP": "Mint In Package",
    "MOC": "Mint On Card",
    "HTH": "Hope That Helps",
    "TIA": "Thanks In Advance",
    "FBA": "Fulfillment By Amazon",
    "FBO": "For Best Offer",
    "FPM": "First Person Mention",
    "Q&A": "Question and Answer",
    "PSA": "Public Service Announcement",
    "ETA": "Estimated Time of Arrival",
    "PS": "Postscript",
    "ICYMI": "In Case You Missed It",
    "DW": "Deal With",
    "BTS": "Behind The Scenes",
    "EOTD": "End Of The Day",
    "BTW": "By The Way",
    "DIY": "Do It Yourself",
    "CR": "Customer Review",
    "FAQ": "Frequently Asked Questions",
    "ASIN": "Amazon Standard Identification Number",
    "BOPIS": "Buy Online, Pick Up In Store",
    "BOGO": "Buy One, Get One",
    "EPC": "Earn Per Click",
    "UPC": "Universal Product Code",
    "ROI": "Return On Investment",
    "KPI": "Key Performance Indicator",
    "CTR": "Click-Through Rate",
    "CPC": "Cost Per Click",
    "CPM": "Cost Per Thousand Impressions",
    "CTA": "Call To Action",
    "PPC": "Pay-Per-Click",
    "CRO": "Conversion Rate Optimization",
    "SERP": "Search Engine Results Page",
    "SKU": "Stock Keeping Unit",
    "SOP": "Standard Operating Procedure",
    "CRaP": "Can't Realize any Profit",
    "LTL": "Less Than Load",
    "OOS": "Out of Stock",
    "P&L": "Profit and Loss",
    "QA": "Quality Assurance",
    "QC": "Quality Control",
    "RFQ": "Request For Quotation",
    "SLA": "Service Level Agreement",
    "SRM": "Supplier Relationship Management",
    "WMS": "Warehouse Management System",
    "EDI": "Electronic Data Interchange",
    "TMS": "Transportation Management System",
    "SCM": "Supply Chain Management",
    "BOM": "Bill Of Materials",
    "BOL": "Bill Of Lading",
    "BOA": "Bill Of Activities",
    "MPS": "Master Production Schedule",
    "MRP": "Material Requirements Planning",
    "ERP": "Enterprise Resource Planning",
    "MRO": "Maintenance, Repair, and Operations",
    "ROP": "Reorder Point",
    "ROQ": "Reorder Quantity",
    "VMI": "Vendor Managed Inventory",
    "ABC": "Always Be Closing",
    "COGS": "Cost Of Goods Sold",
    "DRP": "Distribution Requirements Planning",
    "JIT": "Just In Time",
    "LIFO": "Last In, First Out",
    "EOQ": "Economic Order Quantity",
    "MOQ": "Minimum Order Quantity",
    "SKU": "Stock Keeping Unit",
    "TCO": "Total Cost of Ownership",
    "DTC": "Direct-To-Consumer",
    "OMS": "Order Management System",
    "POS": "Point Of Sale",
    "ROP": "Reorder Point",
    "EOQ": "Economic Order Quantity",
    "CPG": "Consumer Packaged Goods",
    "DMU": "Decision Making Unit",
    "MDF": "Market Development Fund",
    "CRM": "Customer Relationship Management",
    "CLV": "Customer Lifetime Value",
    "LTV": "Lifetime Value",
    "GTM": "Go-To-Market",
    "ROMI": "Return On Marketing Investment",
    "SEO": "Search Engine Optimization",
    "SEM": "Search Engine Marketing",
    "SMM": "Social Media Marketing",
    "CRO": "Conversion Rate Optimization",
    "CTR": "Click-Through Rate",
    "CPA": "Cost Per Acquisition",
    "CPL": "Cost Per Lead",
    "CPS": "Cost Per Sale",
    "CPI": "Cost Per Install",
    "CPV": "Cost Per View",
    "CPC": "Cost Per Click",
    "CPM": "Cost Per Mille",
    "ROAS": "Return On Advertising Spend",
    "CTR": "Click Through Rate",
    "CPC": "Cost Per Click",
    "CR": "Conversion Rate",
    "CPA": "Cost Per Acquisition",
    "LTV": "Lifetime Value",
    "RFM": "Recency, Frequency, Monetary",
    "CVR": "Conversion Rate",
    "MQL": "Marketing Qualified Lead",
    "SQL": "Sales Qualified Lead",
    "CQL": "Customer Qualified Lead",
    "B2B": "Business To Business",
    "B2C": "Business To Consumer",
    "B2G": "Business To Government",
    "C2C": "Consumer To Consumer",
    "CMO": "Chief Marketing Officer",
    "CEO": "Chief Executive Officer",
    "CFO": "Chief Financial Officer",
    "COO": "Chief Operating Officer",
    "CTO": "Chief Technology Officer",
    "CXO": "Chief Experience Officer",
    "CSO": "Chief Sales Officer",
    "CIO": "Chief Information Officer"}

def expand_acronyms(text):
    for acronym, expanded in acronyms.items():
        text = text.replace(acronym, expanded)
    return text

# e. Lemmatization
def lemmatize_text(text):
    lemmatized_tokens = [lemmatizer.lemmatize(word, wordnet.VERB) for word in text.split()]
    return ' '.join(lemmatized_tokens)

# f. Text Normalizer
def normalize_text(text):
    text = remove_accented_chars(text)
    text = expand_acronyms(text)
    return text


df_rating_review['cleaned_review'] = df_rating_review['reviewText'].apply(normalize_text)

/home/orchid/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
/home/orchid/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/home/orchid/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the buil

In [19]:
import re
import nltk
from nltk.corpus import stopwords
def cleaning(data):
    corpus = []
    for i in range(0, len(data)):
        sentence = re.sub('[^a-zA-Z]', ' ', str(data[i])) # d. Removing Special Characters
        sentence = sentence.lower() # Lowering the text
        sentence = sentence.split() # Tokenization
        
        # Remove blank space tokens
        sentence = [word for word in sentence if word.strip()]  # Removes empty strings
        
        all_stopwords = stopwords.words('english') # Removing the stopwords
        #all_stopwords.remove('not')
        # e. Lemmatization
        sentence = [lemmatizer.lemmatize(word) for word in sentence if not word in set(all_stopwords)]
        sentence = ' '.join(sentence)
        corpus.append(sentence)
      
    return corpus

In [20]:
df_rating_review['cleaned_review'] =  cleaning(df_rating_review['cleaned_review'])
df_rating_review.head()

,overall,reviewText,cleaned_review
0,5.0,Great headphones. It's just the cord is too sh...,great headphone cord short
1,5.0,Really like these headphone. Wanted something...,really like headphone wanted something office ...
2,1.0,Wire to headphone broke off in less than a mon...,wire headphone broke le month buy
3,3.0,Very good,good
4,1.0,Currently returning this product because the s...,currently returning product sound come speaker...


## 6. To extract relevant statistics, perform the following EDA -

In [21]:
df.head()

,asin,overall,reviewText,style,reviewerID,unixReviewTime,brand
0,4126895493,5.0,Great headphones. It's just the cord is too sh...,{'Color:': ' Blue W/Mic'},A38RQFVQ1AKJQQ,1424563200,HeadGear
1,4126895493,5.0,Really like these headphone. Wanted something...,{'Color:': ' Blue Zebra W/Mic'},A299MRB9O6GWDE,1494201600,HeadGear
2,4126895493,1.0,Wire to headphone broke off in less than a mon...,{'Color:': ' Blue W/Mic'},A3ACFC6DQQLIQT,1478304000,HeadGear
3,4126895493,3.0,Very good,{'Color:': ' Green'},A36BC0YFDBNB5X,1474675200,HeadGear
4,4126895493,1.0,Currently returning this product because the s...,{'Color:': ' Violet Purple'},A212PQ0HQPNNWM,1468713600,HeadGear


In [22]:
df.isnull().sum()

asin               0
overall            0
reviewText        96
style              0
reviewerID         0
unixReviewTime     0
brand              0
dtype: int64

In [23]:
df_brand = df[['asin', 'overall', 'reviewText', 'brand']]
df_brand = df_brand.dropna()

In [24]:
df_brand.isnull().sum()

asin          0
overall       0
reviewText    0
brand         0
dtype: int64

In [25]:
df_brand['brand'].nunique()

3306

In [26]:
# a. Top 20 most reviewed brands in Headphones
print("Top 20 most reviewed brands:")
df_brand['brand'].value_counts().head(20)

Top 20 most reviewed brands:


Sony              37455
Sennheiser        22976
Plantronics       11946
Bose              11582
Panasonic          8519
Skullcandy         7897
Mpow               7605
JLAB               7513
Roku               7469
JVC                7114
TaoTronics         7065
Samsung            7060
Audio-Technica     7056
Philips            7051
Koss               7002
Kinivo             6444
Apple              6372
Etre Jeune         5969
AmazonBasics       5897
LG                 5695
Name: brand, dtype: int64

In [27]:
# b. Top 20 least reviewed brands in Headphones
print("Top 20 least reviewed brands:")
df_brand['brand'].value_counts().tail(20)

Top 20 least reviewed brands:


Amphony                    5
1 BY 1                     5
SmartLife                  5
Skoloo                     5
TechElec                   5
IFOXTEK                    5
tech21                     5
CZM Tech                   5
California Cable Market    4
SmartDisk                  4
SOUND-SQUARED CO.          4
YooZoo                     4
Replug                     4
DetectorPro                4
NOIZY Brands               3
DSI                        3
Zelco Industries, Inc      3
Fred & Friends             3
Honda                      1
Digital Antenna            1
Name: brand, dtype: int64

In [28]:
# c. Which is the most positively reviewed ‘Headphone’
brand_avg_rating = df.groupby('brand')['overall'].mean()
most_positively_reviewed_brand = brand_avg_rating.idxmax()
highest_avg_rating = brand_avg_rating.max()
print("Most positively reviewed headphone brand:", most_positively_reviewed_brand)
print("Average rating score:", highest_avg_rating)

Most positively reviewed headphone brand: 4 in 1 Charger
Average rating score: 5.0


In [29]:
# d. Show the count of ratings for the product over 5 consecutive years.
df['review_year'] = pd.to_datetime(df['unixReviewTime'], unit='s').dt.year

df.groupby(df['review_year']).size().tail()


review_year
2014     93590
2015    145260
2016    143320
2017     80955
2018     32366
dtype: int64